#### Importing required libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import variation

import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Settings
pd.options.display.float_format = '{:.2f}'.format

#### Importing data

In [2]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')
stores = pd.read_csv('Data/stores.csv')
transactions = pd.read_csv('Data/transactions.csv')
holidays_events = pd.read_csv('Data/holidays_events.csv')
oil = pd.read_csv('Data/oil.csv')
sample_submission = pd.read_csv('Data/sample_submission.csv')

In [3]:
print(len(train))
train.head()

3000888


,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0
1,1,2013-01-01,1,BABY CARE,0.00,0
2,2,2013-01-01,1,BEAUTY,0.00,0
3,3,2013-01-01,1,BEVERAGES,0.00,0
4,4,2013-01-01,1,BOOKS,0.00,0


In [4]:
print(len(test))
test.head()

28512


,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [5]:
print(len(sample_submission)) # same length as test
sample_submission.head()

28512


,id,sales
0,3000888,0.00
1,3000889,0.00
2,3000890,0.00
3,3000891,0.00
4,3000892,0.00


In [6]:
print(len(stores))
stores.head()

54


,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [7]:
print(len(transactions))
transactions.head()

83488


,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [8]:
print(len(holidays_events)) # same length as test
holidays_events.head()

350


,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [9]:
print(len(oil)) # same length as test
oil.head()

1218


,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


#### Ideas
- Can expect correlation between sales and onpromotion (number of items in product family under promotion on given date). More the number of products on promotion in family greater the sales
- Can expect correlation between sales and oil prices
- Is not a holiday: 'Transferred' (Column transferred True), 'Work Day'; Is a Holiday: 'Holiday', 'Transfer', 'Additional', 'Bridge'
- Can add inidcator for 15th and last day of month since wages are paid
- Add indicator or remove weeks after earthquake April 16, 2016

#### Processing data

##### Processing holiday and events data

In [10]:
holidays_events_proc = holidays_events.copy()
holidays_events_proc.head(2)

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False


In [11]:
holidays_events_proc['type'].unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'],
      dtype=object)

In [12]:
# Event is not a holiday. It includes Earthquake, Black friday, etc.
holidays_events_proc[holidays_events_proc['type']=='Event'].reset_index(drop=True).iloc[16:23]

,date,type,locale,locale_name,description,transferred
16,2014-11-28,Event,National,Ecuador,Black Friday,False
17,2014-12-01,Event,National,Ecuador,Cyber Monday,False
18,2015-05-10,Event,National,Ecuador,Dia de la Madre,False
19,2015-11-27,Event,National,Ecuador,Black Friday,False
20,2015-11-30,Event,National,Ecuador,Cyber Monday,False
21,2016-04-16,Event,National,Ecuador,Terremoto Manabi,False
22,2016-04-17,Event,National,Ecuador,Terremoto Manabi+1,False


In [13]:
print('Locale values:', holidays_events_proc['locale'].unique())
print(holidays_events_proc[['locale', 'locale_name']].drop_duplicates()['locale'].value_counts())
# 4 Regions, 19 locales are present in holidays file

print('Levels at which Events are present:', 
holidays_events_proc[holidays_events_proc['type']=='Event']['locale'].unique())
# All events are at National level only

holidays_events_proc[['locale', 'locale_name']].drop_duplicates().sort_values(by=['locale'])

Locale values: ['Local' 'Regional' 'National']
Local       19
Regional     4
National     1
Name: locale, dtype: int64
Levels at which Events are present: ['National']


,locale,locale_name
0,Local,Manta
30,Local,Loja
28,Local,Quito
18,Local,Quevedo
17,Local,Ibarra
16,Local,Ambato
13,Local,Esmeraldas
12,Local,Cayambe
32,Local,Salinas
10,Local,Santo Domingo


In [14]:
events_list = holidays_events_proc[(holidays_events_proc['type']=='Event')]['description'].unique().tolist()

# Changing all strings to lower case
events_list = [str.lower(i) for i in events_list]

# Naming different days of football match the same - Based on description Ecuador’s 
# matches were only on 3 of the days.
events_list = ['mundial de futbol brasil' if ('mundial de futbol brasil' in i) else i for i in events_list]

# Removing suffixes (-1, -2, +1, etc.) for days before and after events
events_list = [i[:i.rfind('-')] if ('-' in i) else (
    i[:i.rfind('+')]  if ('+' in i) else i) for i in events_list]

# 'terremoto manabi' - Earthquake
events_list = np.unique(events_list)
print('Number of unique events:', len(events_list), '\n', events_list)

Number of unique events: 5 
 ['black friday' 'cyber monday' 'dia de la madre'
 'mundial de futbol brasil' 'terremoto manabi']


In [15]:
holidays_events_proc['description'].isna().sum()

0

In [16]:
# Adding flag for Event
holidays_events_proc['Event'] = holidays_events_proc['type'] == 'Event'

# Repeating same steps as above
holidays_events_proc['Event_name_base'] = holidays_events_proc['description'].str.lower()
holidays_events_proc['Event_name_base'] = holidays_events_proc['Event_name_base'].map(
    lambda i: 'mundial de futbol brasil' if ('mundial de futbol brasil' in i) else i) 
holidays_events_proc['Event_name_base'] = holidays_events_proc['Event_name_base'].map(
    lambda i: i[:i.rfind('-')] if ('-' in i) else (
    i[:i.rfind('+')]  if ('+' in i) else i)) 
holidays_events_proc['Event_name_base'] = np.where(
    holidays_events_proc['Event']==True, 
    holidays_events_proc['Event_name_base'], None)

holidays_events_proc[holidays_events_proc['type']=='Event'].reset_index(drop=True).iloc[16:18]

,date,type,locale,locale_name,description,transferred,Event,Event_name_base
16,2014-11-28,Event,National,Ecuador,Black Friday,False,True,black friday
17,2014-12-01,Event,National,Ecuador,Cyber Monday,False,True,cyber monday


Check 'cantonizacion', 'recupero', 'dia', 'fundacion', 'primer', 'independencia', 'provincializacion'
-  'cantonizacion' - occurs as 'cantonizacion' de 'locale_name'
-  'fundacion' - occurs as 'fundacion' de 'locale_name'
-  'provincializacion' - occurs as 'provincializacion' de 'locale_name'
-  'independencia' - occurs as 'independencia' de 'locale_name' and independencia de Cuenca/ Guayaquil at National level but Cuenca, Guayaquil are also locale names
- Appears like 'puente' is bridge and 'recupero puente' is work day compensating for bridge
-  'dia', 'primer' - All at national level. no other pattern observed

In [17]:
holidays_list = holidays_events_proc[(holidays_events_proc['Event']!=True) & 
(holidays_events_proc['transferred']==False) & 
(holidays_events_proc['type']!='Work Day')]['description'].unique().tolist()

# Changing all strings to lower case
holidays_list = [str.lower(i) for i in holidays_list]

# Removing prefix 'traslado' for transferred holidays
holidays_list = [i[len('traslado '):] if ('traslado' in i) else i for i in holidays_list]

# Removing prefix 'puente' for bridge holidays
holidays_list = [i[len('puente '):] if ('puente' in i) else i for i in holidays_list]

# Removing prefix 'Traslado' for transferred holidays
holidays_list = [i[len('recupero puente '):] if ('recupero puente' in i) else i for i in holidays_list]

# Removing suffixes (-1, -2, +1, etc.) for days before and after holidays
holidays_list = [i[:i.rfind('-')] if ('-' in i) else (
    i[:i.rfind('+')]  if ('+' in i) else i) for i in holidays_list]


holidays_list = np.unique(holidays_list)
print('Number of unique holidays:', len(holidays_list), '\n', holidays_list)

Number of unique holidays: 38 
 ['batalla de pichincha' 'cantonizacion de cayambe'
 'cantonizacion de el carmen' 'cantonizacion de guaranda'
 'cantonizacion de latacunga' 'cantonizacion de libertad'
 'cantonizacion de quevedo' 'cantonizacion de riobamba'
 'cantonizacion de salinas' 'cantonizacion del puyo' 'carnaval'
 'dia de difuntos' 'dia de la madre' 'dia del trabajo'
 'fundacion de ambato' 'fundacion de cuenca' 'fundacion de esmeraldas'
 'fundacion de guayaquil' 'fundacion de ibarra' 'fundacion de loja'
 'fundacion de machala' 'fundacion de manta' 'fundacion de quito'
 'fundacion de riobamba' 'fundacion de santo domingo'
 'independencia de ambato' 'independencia de cuenca'
 'independencia de guaranda' 'independencia de guayaquil'
 'independencia de latacunga' 'navidad' 'primer dia del ano'
 'primer grito de independencia' 'provincializacion de cotopaxi'
 'provincializacion de imbabura' 'provincializacion de santo domingo'
 'provincializacion santa elena' 'viernes santo']


In [18]:
# Adding holiday flag
holidays_events_proc['Holiday'] = (
    (holidays_events_proc['Event']!=True) & 
    (holidays_events_proc['transferred']==False) & 
    (holidays_events_proc['type']!='Work Day'))

# Repeating same steps as above
holidays_events_proc['Holiday_name_base'] =  holidays_events_proc['description'].str.lower()
holidays_events_proc['Holiday_name_base'] = holidays_events_proc['Holiday_name_base'].map(
    lambda i: i[len('traslado '):] if ('traslado' in i) else i)
holidays_events_proc['Holiday_name_base'] = holidays_events_proc['Holiday_name_base'].map(
    lambda i: i[len('puente '):] if ('puente' in i) else i) 
holidays_events_proc['Holiday_name_base'] = holidays_events_proc['Holiday_name_base'].map(
    lambda i: i[len('recupero puente '):] if ('recupero puente' in i) else i) 
holidays_events_proc['Holiday_name_base'] = holidays_events_proc['Holiday_name_base'].map(
    lambda i: i[:i.rfind('-')] if ('-' in i) else (
    i[:i.rfind('+')]  if ('+' in i) else i)) 
holidays_events_proc['Holiday_name_base'] =  np.where(
    holidays_events_proc['Holiday']==True, 
    holidays_events_proc['Holiday_name_base'], None)

holidays_events_proc.head(3)

,date,type,locale,locale_name,description,transferred,Event,Event_name_base,Holiday,Holiday_name_base
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,False,None,True,fundacion de manta
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,False,None,True,provincializacion de cotopaxi
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,False,None,True,fundacion de cuenca


##### Processing and exploring train data set

In [19]:
# Exploring train data set
train['date'] = pd.to_datetime(train['date'])
data_start_date = train['date'].min()
data_end_date = train['date'].max()
print(data_start_date, data_end_date)
train.head()

2013-01-01 00:00:00 2017-08-15 00:00:00


,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0
1,1,2013-01-01,1,BABY CARE,0.00,0
2,2,2013-01-01,1,BEAUTY,0.00,0
3,3,2013-01-01,1,BEVERAGES,0.00,0
4,4,2013-01-01,1,BOOKS,0.00,0


In [20]:
# 75% of sales values are below 200 but maximum sales is 100 thousand
# Average number of items on promotion is 2 and 75% of onpromotion values is 0, maximum is 741 items
train[['sales', 'onpromotion']].describe()

,sales,onpromotion
count,3000888.00,3000888.00
mean,357.78,2.60
std,1102.00,12.22
min,0.00,0.00
25%,0.00,0.00
50%,11.00,0.00
75%,195.85,0.00
max,124717.00,741.00


In [21]:
# Every store-product combination present in train data
print('Unique store count:', train['store_nbr'].nunique(), '\n', 
'Unique product family count:', train['family'].nunique(),  '\n', 
'Unique store, product family combinations:', 
train[['store_nbr', 'family']].drop_duplicates().shape[0])
54*33


Unique store count: 54 
 Unique product family count: 33 
 Unique store, product family combinations: 1782


1782

In [22]:
# Column indicating store, product family combination
train_proc = train.copy()
train_proc['comb'] = train_proc['store_nbr'].map(int).map(str) + '_' + train_proc['family']
train_proc.head(2)

,id,date,store_nbr,family,sales,onpromotion,comb
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0,1_AUTOMOTIVE
1,1,2013-01-01,1,BABY CARE,0.00,0,1_BABY CARE


In [23]:
# Separating combinations with no sales so that they can directly be assinged as zero
no_sales_combs = list(set(train_proc['comb'].unique()) - 
set(train_proc[train_proc['sales']!=0]['comb'].unique()))
print('Count of combinations with no sales:', len(no_sales_combs))

Count of combinations with no sales: 53


##### Demand type classification
![](demand_classification.png)
- Smooth demand (ADI < 1.32 and CV² < 0.49). The demand is very regular in time and in quantity. It is therefore easy to forecast and you won’t have trouble reaching a low forecasting error level. 
- Intermittent demand (ADI >= 1.32 and CV² < 0.49). The demand history shows very little variation in demand quantity but a high variation in the interval between two demands. Though specific forecasting methods tackle intermittent demands, the forecast error margin is considerably higher.
- Erratic demand (ADI < 1.32 and CV² >= 0.49). The demand has regular occurrences in time with high quantity variations. Your forecast accuracy remains shaky.
- Lumpy demand (ADI >= 1.32 and CV² >= 0.49). The demand is characterized by a large variation in quantity and in time. It is actually impossible to produce a reliable forecast, no matter which forecasting tools you use. This particular type of demand pattern is unforecastable.

In [24]:
# Demand classification, Missing days count, Years of data present

def comb_info_calc(df):
    """" Function calculates duration, count of missing dates, adi, cv^2.
    df: data frame with date and sales columns
    """
    # ADI 
    df1 = df[df['sales']!=0].copy()
    df_act = df[df['date']>=df1['date'].min()].copy()
    days_pres = len(df_act)
    df_all_dates = pd.DataFrame({'date': pd.date_range(
        start=df1['date'].min(), end=data_end_date)})
    df_act = df_act.merge(df_all_dates, on=['date'], how='right')
    days_tot = len(df_act)
    df_act['sales'] = df_act['sales'].fillna(0)
    adi = len(df_all_dates)/df_act[df_act['sales']!=0].shape[0]

    # CV
    cv2 = variation(df_act['sales'])
    return([days_tot, days_tot-days_pres, adi, cv2])

# Keeping only combs with sales
train_proc_sales = train_proc[train_proc['comb'].isin(no_sales_combs)==False].copy()

comb_info_tb = train_proc_sales.groupby('comb').apply(
    comb_info_calc).reset_index().rename(columns={0: 'COMB_INFO'})
comb_info_tb['Total_days'] = comb_info_tb['COMB_INFO'].map(lambda x: x[0])
comb_info_tb['Missing_days'] = comb_info_tb['COMB_INFO'].map(lambda x: x[1])

comb_info_tb['Years_of_data'] = comb_info_tb['Total_days']/365

# Calculating adi, cv^2 and demand classification
comb_info_tb['ADI'] = comb_info_tb['COMB_INFO'].map(lambda x: x[2])
comb_info_tb['CV2'] = comb_info_tb['COMB_INFO'].map(lambda x: x[3])
comb_info_tb.drop(columns=['COMB_INFO'], inplace=True)

comb_info_tb['Demand_classification'] = np.where(
    (comb_info_tb['ADI']<1.32) & (comb_info_tb['CV2']<0.49), 'Smooth',
    np.where((comb_info_tb['ADI']>=1.32) & (comb_info_tb['CV2']<0.49), 'Intermittent',
    np.where((comb_info_tb['ADI']<1.32) & (comb_info_tb['CV2']>=0.49), 'Erratic', 'Lumpy'))
)

print(comb_info_tb['Demand_classification'].value_counts().reset_index())
comb_info_tb.head()

     index  Demand_classification
0  Erratic                    797
1   Smooth                    517
2    Lumpy                    415


,comb,Total_days,Missing_days,Years_of_data,ADI,CV2,Demand_classification
0,10_AUTOMOTIVE,1687,4,4.62,1.14,0.86,Erratic
1,10_BABY CARE,1142,3,3.13,6.92,3.41,Lumpy
2,10_BEAUTY,1687,4,4.62,1.47,1.03,Lumpy
3,10_BEVERAGES,1687,4,4.62,1.00,0.47,Smooth
4,10_BREAD/BAKERY,1687,4,4.62,1.00,0.33,Smooth


In [25]:
# 75% of the combinations have >=3.5 years of data, Maximum number of dates missing is 4
comb_info_tb[['Years_of_data', 'Missing_days', 'Total_days']].describe()

,Years_of_data,Missing_days,Total_days
count,1729.00,1729.00,1729.00
mean,3.88,3.37,1417.29
std,1.06,0.90,387.89
min,0.10,0.00,37.00
25%,3.62,3.00,1322.00
50%,4.62,4.00,1686.00
75%,4.62,4.00,1687.00
max,4.62,4.00,1688.00


In [26]:
# Is imputing missing weeks with 0 meaningful?

# Seasonality/ trend present in data?

# ACF, PACF